**PART 1: Data Extraction and Feature Extraction with Preprocessing**

In [1]:
'''
1. Pre - Requisite
Install all Library
'''
# 1) Upgrade core tooling inside THIS kernel
%pip install --upgrade pip setuptools wheel --user

# 2) Clean install of audio deps (librosa works with Py3.12 now via numba>=0.61)
%pip install librosa==0.11.0 numba==0.61.2 llvmlite soxr soundfile ffmpeg-python matplotlib tqdm pandas numpy --user

%pip install pandas numpy librosa torchaudio tqdm ffmpeg-python matplotlib soundfile --user
%pip install --upgrade moviepy imageio-ffmpeg torchaudio soundfile numpy tqdm --user

Note: you may need to restart the kernel to use updated packages.


  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.2.6 which is incompatible.
torchvision 0.19.1+cu121 requires torch==2.4.1+cu121, but you have torch 2.8.0 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


  Using cached numpy-2.3.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.3-cp312-cp312-win_amd64.whl (12.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.3.3 which is incompatible.
torchvision 0.19.1+cu121 requires torch==2.4.1+cu121, but you have torch 2.8.0 which is incompatible.
numba 0.61.2 requires numpy<2.3,>=1.24, but you have numpy 2.3.3 which is incompatible.
scipy 1.13.0 requires numpy<2.3,>=1.22.4, but you have numpy 2.3.3 which is incompatible.


In [2]:
import os, sys, json, random, subprocess, shutil
from pathlib import Path
import numpy as np
import pandas as pd
import librosa, librosa.display
import soundfile as sf
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Audio, display
from SoccerNet.Downloader import SoccerNetDownloader
import torchaudio

In [3]:
SN_ROOT = Path(r"C:/Users/CYBORG 15/Documents/GitHub/elec5305-project-520038655/SoccerNet")
SN_ROOT.mkdir(parents=True, exist_ok=True)

SOCCERNET_NDA = 's0cc3rn3t'

# Workspace
WORK_DIR  = Path("./sn_audio_work"); WORK_DIR.mkdir(parents=True, exist_ok=True)
AUDIO_DIR = WORK_DIR / "full_match_audio_wav"; AUDIO_DIR.mkdir(parents=True, exist_ok=True)
CLIPS_DIR = WORK_DIR / "clips";               CLIPS_DIR.mkdir(parents=True, exist_ok=True)
FEAT_DIR  = WORK_DIR / "mel64";               FEAT_DIR.mkdir(parents=True, exist_ok=True)

# Audio + reproducibility
TARGET_SR    = 16000
WINDOWS      = [15, 30]   # 15s/30s event windows
RANDOM_SEED  = 1337
random.seed(RANDOM_SEED)
torchaudio.set_audio_backend("soundfile")  # stable on Windows

print("SN_ROOT:", SN_ROOT)
print("WORK_DIR:", WORK_DIR.resolve())


SN_ROOT: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet
WORK_DIR: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\sn_audio_work


C:\Users\CYBORG 15\AppData\Local\Temp\ipykernel_3516\959740583.py:17: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.set_audio_backend("soundfile")  # stable on Windows
C:\Users\CYBORG 15\AppData\Roaming\Python\Python312\site-packages\torchaudio\_internal\module_utils.py:71: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  return func(*args, **kwargs)


In [4]:
from pathlib import Path
from SoccerNet.Downloader import SoccerNetDownloader
import json

def find_label_files(root: Path):
    files = list(root.rglob("Labels-v2.json"))
    if not files:
        files = list(root.rglob("Labels.json"))
    print(f"Found label files: {len(files)}")
    return files

def _parse_time_mmss(s: str):
    parts = s.split(":")
    try:
        if len(parts)==3:
            hh,mm,ss = parts; return int(hh)*3600 + int(mm)*60 + float(ss)
        if len(parts)==2:
            mm,ss = parts;    return int(mm)*60 + float(ss)
    except Exception:
        return None
    return None

def load_labels_with_half(labels_path: Path):
    """Return list of dicts: {'time_s': float, 'label': str, 'half': 0|1|2}"""
    with open(labels_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    ann = data.get("annotations") or data.get("events") or data.get("labels") or []
    out=[]
    for ev in ann:
        lab  = ev.get("label") or ev.get("event") or ev.get("type")
        if not lab:
            continue
        half = ev.get("half")
        if half not in (1,2):
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                try: half = int(gt.split(" - ")[0].strip())
                except: half = None
        pos = ev.get("position")
        t = None
        if isinstance(pos,(int,float)): t = float(pos)
        elif isinstance(pos,str):       t = _parse_time_mmss(pos)
        if t is None:
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                t = _parse_time_mmss(gt.split(" - ")[1].strip())
        if t is None:
            continue
        out.append({"time_s": float(t), "label": lab, "half": (half if half in (1,2) else 0)})
    return out

In [5]:
def find_half_videos(match_dir: Path):
    """Return {'1': Path|None, '2': Path|None, 'any': Path|None}"""
    half_map = {1: None, 2: None}
    any_vid = None
    preferred = [
        ("1_720p.mkv", 1), ("2_720p.mkv", 2),
        ("1_224p.mkv", 1), ("2_224p.mkv", 2),
        ("1.mkv", 1),     ("2.mkv", 2),
        ("1.mp4", 1),     ("2.mp4", 2),
    ]
    for name, half in preferred:
        p = match_dir / name
        if p.exists():
            half_map[half] = p
            if any_vid is None: any_vid = p
    if any_vid is None:
        for ext in (".mkv",".mp4",".m4v",".mov"):
            found = list(match_dir.glob(f"*{ext}"))
            if found:
                any_vid = found[0]; break
    return {"1": half_map[1], "2": half_map[2], "any": any_vid}


In [6]:
rows=[]
for lf in tqdm(find_label_files(SN_ROOT), desc="Index labels"):
    mdir  = lf.parent
    split = mdir.parent.name
    evs   = load_labels_with_half(lf)
    if not evs:
        continue
    vids  = find_half_videos(mdir)  # may be all None if videos not yet downloaded
    for e in evs:
        # choose video by half when available, else fallback to any
        video = vids[str(e["half"])] if e["half"] in (1,2) else vids["any"]
        rows.append({
            "split": split,
            "match_dir": str(mdir),
            "labels_json": str(lf),
            "event_time_s": e["time_s"],
            "event_label": e["label"],
            "half": int(e["half"]),
            "video_path": (str(video) if video is not None else None)
        })

df_events = pd.DataFrame(rows).drop_duplicates(
    subset=["match_dir","event_time_s","event_label","half"]
).reset_index(drop=True)

print("Indexed events:", len(df_events))
print("Have video_path for:", df_events["video_path"].notna().sum(), "events")
ALL_LABELS = sorted(df_events["event_label"].unique())
print("Classes:", ALL_LABELS, "| total:", len(ALL_LABELS))


Found label files: 95


Index labels: 100%|██████████| 95/95 [00:00<00:00, 278.64it/s]

Indexed events: 20428
Have video_path for: 9814 events
Classes: ['Ball out of play', 'Clearance', 'Corner', 'Direct free-kick', 'Foul', 'Goal', 'Indirect free-kick', 'Kick-off', 'Offside', 'Penalty', 'Red card', 'Shots off target', 'Shots on target', 'Substitution', 'Throw-in', 'Yellow card', 'Yellow->red card'] | total: 17


In [7]:
from SoccerNet.Downloader import SoccerNetDownloader
from pathlib import Path

# Create an instance, pointing to your dataset root
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory=str(SN_ROOT))
mySoccerNetDownloader.password = SOCCERNET_NDA  # 's0cc3rn3t'

print("SoccerNet root:", SN_ROOT)

# ✅ Download labels (small files, safe)
mySoccerNetDownloader.downloadDataTask(task="labels", split=["train","valid","test","challenge"])

# ✅ Download 224p halves, but exclude spain_laliga
splits = ["train","valid","test","challenge"]
for s in splits:
    split_dir = Path(SN_ROOT) / s
    if not split_dir.exists():
        continue
    for comp_dir in split_dir.iterdir():
        if not comp_dir.is_dir():
            continue
        if "spain_laliga" in comp_dir.name:
            print(f"⏭️ Skipping competition: {comp_dir}")
            continue
        print(f"⬇️ Downloading videos for competition: {comp_dir}")
        mySoccerNetDownloader.downloadGames(
            files=["1_224p.mkv", "2_224p.mkv"],
            split=[s]
        )
# Create an instance, pointing to your dataset root
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory=str(SN_ROOT))
mySoccerNetDownloader.password = SOCCERNET_NDA  # 's0cc3rn3t'

print("SoccerNet root:", SN_ROOT)

# ✅ Ensure labels are present (small JSON files, will skip if already exist)
mySoccerNetDownloader.downloadDataTask(task="labels", split=["train","valid","test","challenge"])

# Instead: scan and list what’s already downloaded
video_exts = {".mkv", ".mp4", ".m4v", ".mov"}
downloaded_videos = []
for f in SN_ROOT.rglob("*"):
    if f.suffix in video_exts and f.name.startswith(("1_", "2_")):
        downloaded_videos.append(f)

print(f"[Scan] Total video halves found: {len(downloaded_videos)}")
if downloaded_videos:
    # Show just the first 10 to avoid huge spam
    for v in sorted(downloaded_videos)[:10]:
        print(" -", v.relative_to(SN_ROOT))
    if len(downloaded_videos) > 10:
        print(f"... and {len(downloaded_videos)-10} more")
else:
    print("⚠️ No video halves found. Check your dataset folder.")

SoccerNet root: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet
ERROR Unknown task: labels
SoccerNet root: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet
ERROR Unknown task: labels
[Scan] Total video halves found: 95
 - england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_224p.mkv
 - england_epl\2014-2015\2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal\1_224p.mkv
 - england_epl\2014-2015\2015-02-21 - 18-00 Swansea 2 - 1 Manchester United\1_224p.mkv
 - england_epl\2014-2015\2015-02-22 - 19-15 Southampton 0 - 2 Liverpool\1_224p.mkv
 - england_epl\2014-2015\2015-04-11 - 19-30 Burnley 0 - 1 Arsenal\1_224p.mkv
 - england_epl\2014-2015\2015-05-17 - 18-00 Manchester United 1 - 1 Arsenal\1_224p.mkv
 - england_epl\2015-2016\2015-08-08 - 19-30 Chelsea 2 - 2 Swansea\1_224p.mkv
 - england_epl\2015-2016\2015-08-16 - 18-00 Manchester City 3 - 0 Chelsea\1_224p.mkv
 - england_epl\2015-2016\2015-08-23 - 15-30 West Brom 2 - 3 Chelsea\1_224p.

In [8]:
from pathlib import Path
import re, shutil
import pandas as pd
from SoccerNet.Downloader import SoccerNetDownloader

# --- config ---
NEEDED_FILES = ["1_224p.mkv", "2_224p.mkv"]
DROP_COMP     = "spain_laliga"
CANON_COMPS   = [
    "england_epl",
    "europe_uefa-champions-league",
    "france_ligue-1",
    "germany_bundesliga",
    "italy_serie-a",
]

# --- helper: parse competition/season from your on-disk layout ---
SEASON_RE = re.compile(r"^\d{4}-\d{4}$")
def find_label_game_dirs(root: Path):
    return [p.parent for p in root.rglob("Labels*.json")]

def parse_comp_season_from_game_dir(game_dir: Path, root: Path):
    parts = game_dir.relative_to(root).parts
    comp = None; season = None
    for i, p in enumerate(parts):
        if p in (set(CANON_COMPS) | {DROP_COMP}):
            comp = p
            if i+1 < len(parts) and SEASON_RE.match(parts[i+1]):
                season = parts[i+1]
            break
    if comp is None and len(parts) >= 1:
        comp = parts[0]
    if season is None:
        for p in parts:
            if SEASON_RE.match(p): season = p; break
    return comp, season

def list_kept_game_dirs(root: Path, keep_comps: list[str]):
    out = []
    for gd in find_label_game_dirs(root):
        comp, _ = parse_comp_season_from_game_dir(gd, root)
        if comp in keep_comps:
            out.append(gd)
    return out

def check_missing_halves(game_dirs: list[Path], need_files=NEEDED_FILES):
    missing = []
    have = 0
    for g in game_dirs:
        miss = [f for f in need_files if not (g / f).exists()]
        if miss: missing.append((g, miss))
        else:    have += 1
    return have, missing

# --- 0) Make a downloader instance (for labels if ever needed) ---
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory=str(SN_ROOT))
mySoccerNetDownloader.password = SOCCERNET_NDA

print("SoccerNet root:", SN_ROOT)

# --- 1) Ensure labels exist (fast & small) ---
# Safe to call repeatedly; it skips existing files.
mySoccerNetDownloader.downloadDataTask(task="labels", split=["train","valid","test","challenge"])

# --- 2) Inventory competitions present from labels ---
label_games = find_label_game_dirs(SN_ROOT)
print(f"[Labels] Games with labels found: {len(label_games)}")

inventory = {}
for gd in label_games:
    comp, season = parse_comp_season_from_game_dir(gd, SN_ROOT)
    inventory.setdefault(comp, set())
    if season: inventory[comp].add(season)

# Warn if LaLiga exists; delete or ignore as you wish (here: just ignore printing)
if DROP_COMP in inventory and len(inventory[DROP_COMP]) > 0:
    print(f"⚠️ Found '{DROP_COMP}' labels on disk. They will be excluded from processing.")
else:
    print("✅ 'spain_laliga' not present — nothing to exclude.")

# --- 3) Build kept list (exclude LaLiga; keep canonical comps that exist) ---
KEEP_COMPETITIONS = [c for c in CANON_COMPS if c in inventory]
print("Keeping competitions:", KEEP_COMPETITIONS)

# --- 4) Filter to kept games and check which are complete (both halves) ---
keep_game_dirs = list_kept_game_dirs(SN_ROOT, KEEP_COMPETITIONS)
have_count, missing_list = check_missing_halves(keep_game_dirs, NEEDED_FILES)

print(f"[Scan] Kept games (labels): {len(keep_game_dirs)}")
print(f"[Scan] Complete (both halves present): {have_count}")
print(f"[Scan] Incomplete/missing: {len(missing_list)}")

# --- 5) Use ONLY the complete games; no further downloads requested ---
complete_games = [g for g in keep_game_dirs if all((g/f).exists() for f in NEEDED_FILES)]

# Small assertion/print
if len(complete_games) == 0:
    print("❌ No complete games found. If this is unexpected, double-check paths or consider downloading missing halves.")
else:
    print(f"✅ All data ready: using {len(complete_games)} complete games (excluding '{DROP_COMP}').")

# --- 6) Save list of complete game directories for downstream preprocessing ---
complete_df = pd.DataFrame({"game_dir": [str(g) for g in complete_games]})
COMPLETE_LIST_CSV = (WORK_DIR / "complete_games_224p.csv")
complete_df.to_csv(COMPLETE_LIST_CSV, index=False)
print("Saved list of complete games to:", COMPLETE_LIST_CSV)


SoccerNet root: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet
ERROR Unknown task: labels
[Labels] Games with labels found: 190
✅ 'spain_laliga' not present — nothing to exclude.
Keeping competitions: ['england_epl']
[Scan] Kept games (labels): 190
[Scan] Complete (both halves present): 0
[Scan] Incomplete/missing: 190
❌ No complete games found. If this is unexpected, double-check paths or consider downloading missing halves.
Saved list of complete games to: sn_audio_work\complete_games_224p.csv


In [9]:
from pathlib import Path

def find_label_game_dirs(root: Path):
    """Return list of game dirs that contain Labels*.json."""
    games = []
    for p in root.rglob("Labels*.json"):
        # game dir = parent of label file
        games.append(p.parent)
    return games

def list_competitions_from_labels(root: Path):
    """Return sorted list of (split, competition) tuples present on disk."""
    comps = set()
    for game_dir in find_label_game_dirs(root):
        # Expect: root / split / competition / season / game
        parts = game_dir.relative_to(root).parts
        if len(parts) >= 2:
            split, competition = parts[0], parts[1]
            comps.add((split, competition))
    return sorted(comps)

label_games = find_label_game_dirs(SN_ROOT)
print(f"Games with labels found: {len(label_games)}")
detected = list_competitions_from_labels(SN_ROOT)
print("Detected (split, competition) present from labels:")
for s,c in detected[:15]:
    print(" -", s, "|", c)
print("Total competitions detected:", len({c for _,c in detected}))


Games with labels found: 190
Detected (split, competition) present from labels:
 - england_epl | 2014-2015
 - england_epl | 2015-2016
 - england_epl | 2016-2017
Total competitions detected: 3


In [10]:
# === Cell: EPL inventory per HALF — available if (labels exist in that half) AND (WAV exists) ===
from pathlib import Path
import re, json
import pandas as pd

# Assumes SN_ROOT, AUDIO_DIR, TARGET_SR, TARGET_LABELS exist from earlier cells.
# Fallbacks (safe defaults) if you ran this cell standalone:
try:
    SN_ROOT
except NameError:
    SN_ROOT = Path(r"C:/path/to/SoccerNet")  # <- set me if running standalone
try:
    WORK_DIR
except NameError:
    WORK_DIR  = Path("./sn_audio_work")
try:
    AUDIO_DIR
except NameError:
    AUDIO_DIR = WORK_DIR / "full_match_audio_wav"
try:
    TARGET_SR
except NameError:
    TARGET_SR = 16000
try:
    TARGET_LABELS
except NameError:
    TARGET_LABELS = {
        'Yellow->red card','Ball out of play','Foul','Indirect free-kick','Penalty','Shots off target',
        'Offside','Yellow card','Red card','Goal','Direct free-kick','Substitution','Clearance',
        'Kick-off','Corner','Throw-in','Shots on target'
    }

TARGET_COMP = "england_epl"
SEASON_RE = re.compile(r"^\d{4}-\d{4}$")

def parse_comp_season_from_game_dir(game_dir: Path, root: Path):
    parts = game_dir.relative_to(root).parts
    comp = None; season = None
    for i, p in enumerate(parts):
        if p == TARGET_COMP:
            comp = p
            if i+1 < len(parts) and SEASON_RE.match(parts[i+1]):
                season = parts[i+1]
            break
    if comp is None and len(parts) >= 1:
        comp = parts[0]
    if season is None:
        for p in parts:
            if SEASON_RE.match(p):
                season = p; break
    return comp, season

def find_label_game_dirs(root: Path):
    """All game dirs (any comp) that contain Labels*.json."""
    return [p.parent for p in root.rglob("Labels*.json")]

def _parse_time_mmss(s: str):
    parts = s.split(":")
    try:
        if len(parts)==3:
            hh,mm,ss = parts; return int(hh)*3600 + int(mm)*60 + float(ss)
        if len(parts)==2:
            mm,ss = parts;    return int(mm)*60 + float(ss)
    except Exception:
        return None
    return None

def load_labels_with_half(labels_path: Path):
    """Return list of dicts: {'time_s': float, 'label': str, 'half': int in {1,2}}."""
    with open(labels_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    ann = data.get("annotations") or data.get("events") or data.get("labels") or []
    out=[]
    for ev in ann:
        lab  = ev.get("label") or ev.get("event") or ev.get("type")
        if not lab:
            continue
        half = ev.get("half")
        if half not in (1,2):
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                try: half = int(gt.split(" - ")[0].strip())
                except: half = 1
        pos = ev.get("position")
        t = None
        if isinstance(pos,(int,float)): t = float(pos)
        elif isinstance(pos,str):       t = _parse_time_mmss(pos)
        if t is None:
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                t = _parse_time_mmss(gt.split(" - ")[1].strip())
        if t is None: 
            continue
        out.append({"time_s": float(t), "label": lab, "half": int(half)})
    return out

def expected_half_wav_path(comp: str, season: str, game: str, half: int, sr: int = TARGET_SR) -> Path:
    # Must match your ensure_half_wav() naming
    return AUDIO_DIR / f"{comp}__{season}__{game}__H{half}_{sr}Hz.wav"

# ---- Build per-half availability ----
label_games = [gd for gd in find_label_game_dirs(SN_ROOT) if parse_comp_season_from_game_dir(gd, SN_ROOT)[0] == TARGET_COMP]
print(f"[Scan] Games with label JSONs under {TARGET_COMP}: {len(label_games)}")

rows = []
missing = []  # halves that have labels but WAV missing

for gd in label_games:
    comp, season = parse_comp_season_from_game_dir(gd, SN_ROOT)
    game = gd.name
    labels_json = (gd / "Labels-v2.json") if (gd / "Labels-v2.json").exists() else (gd / "Labels.json")

    # Which halves have at least one TARGET_LABEL event?
    try:
        evs = load_labels_with_half(labels_json)
    except Exception as e:
        # if label file unparsable, skip this game
        continue
    halves_with_labels = sorted({e["half"] for e in evs if e["label"] in TARGET_LABELS})

    for h in halves_with_labels:
        wav_p = expected_half_wav_path(comp, season, game, h, TARGET_SR)
        if wav_p.exists():
            rows.append({
                "competition": comp,
                "season": season,
                "game": game,
                "game_dir": str(gd),
                "half": h,
                "wav_path": str(wav_p),
                "has_wav": True
            })
        else:
            missing.append((gd, h, wav_p))
            rows.append({
                "competition": comp,
                "season": season,
                "game": game,
                "game_dir": str(gd),
                "half": h,
                "wav_path": str(wav_p),
                "has_wav": False
            })

df_halves = pd.DataFrame(rows).sort_values(["season","game","half"]).reset_index(drop=True)

n_halves_with_labels = len(df_halves)
n_available = int(df_halves["has_wav"].sum())
print(f"[Result] EPL halves with labels: {n_halves_with_labels}")
print(f"[Result] EPL halves READY (labels + WAV present): {n_available}")
print(f"[Result] Missing WAV for {n_halves_with_labels - n_available} halves (show up to 5):")
for gd, h, wp in missing[:5]:
    print(f" - {gd}  H{h}  expected: {wp.name}")

# Optional: keep only the ready halves for downstream use
df_halves_ready = df_halves[df_halves["has_wav"]].reset_index(drop=True)
print(f"\nReady halves DataFrame shape: {df_halves_ready.shape}")
df_halves_ready.head(3)


[Scan] Games with label JSONs under england_epl: 190
[Result] EPL halves with labels: 380
[Result] EPL halves READY (labels + WAV present): 78
[Result] Missing WAV for 302 halves (show up to 5):
 - C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley  H2  expected: england_epl__2014-2015__2015-02-21 - 18-00 Chelsea 1 - 1 Burnley__H2_16000Hz.wav
 - C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley  H2  expected: england_epl__2014-2015__2015-02-21 - 18-00 Chelsea 1 - 1 Burnley__H2_16000Hz.wav
 - C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal  H2  expected: england_epl__2014-2015__2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal__H2_16000Hz.wav
 - C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_ep

,competition,season,game,game_dir,half,wav_path,has_wav
0,england_epl,2014-2015,2015-02-21 - 18-00 Chelsea 1 - 1 Burnley,C:\Users\CYBORG 15\Documents\GitHub\elec5305-p...,1,sn_audio_work\full_match_audio_wav\england_epl...,True
1,england_epl,2014-2015,2015-02-21 - 18-00 Chelsea 1 - 1 Burnley,C:\Users\CYBORG 15\Documents\GitHub\elec5305-p...,1,sn_audio_work\full_match_audio_wav\england_epl...,True
2,england_epl,2014-2015,2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal,C:\Users\CYBORG 15\Documents\GitHub\elec5305-p...,1,sn_audio_work\full_match_audio_wav\england_epl...,True


In [15]:
# === Build clips + mel features + dataset_index.csv (idempotent + atomic saves + corruption repair) ===
import os, re, json, math, numpy as np, pandas as pd, torch, torchaudio, tempfile
from pathlib import Path
from tqdm import tqdm

# ---- Config (reuse if already defined) ----
SN_ROOT   = Path(globals().get("SN_ROOT", r"C:/Users/YourUser/SoccerNet"))      # << set if needed
WORK_DIR  = Path(globals().get("WORK_DIR", "./sn_audio_work"))
AUDIO_DIR = Path(globals().get("AUDIO_DIR", WORK_DIR / "full_match_audio_wav"))
CLIPS_DIR = Path(globals().get("CLIPS_DIR", WORK_DIR / "clips")); CLIPS_DIR.mkdir(parents=True, exist_ok=True)
FEAT_DIR  = Path(globals().get("FEAT_DIR",  WORK_DIR / "mel64"));  FEAT_DIR.mkdir(parents=True, exist_ok=True)
INDEX_CSV = Path(globals().get("INDEX_CSV", WORK_DIR / "dataset_index.csv"))

TARGET_COMP = "england_epl"
TARGET_SR   = int(globals().get("TARGET_SR", 16000))
N_MELS      = int(globals().get("N_MELS", 64))
N_FFT       = int(globals().get("N_FFT", 1024))
HOP         = int(globals().get("HOP", 320))
WINDOWS     = list(globals().get("WINDOWS", [15, 30]))

# Idempotency & repair controls
OVERWRITE_MELS    = False     # force recompute even if present
OVERWRITE_WAVS    = False     # force rewrite clip wavs
REPAIR_CORRUPT    = True      # recompute invalid/partial npy files
VALIDATE_NPY_SHAPE = (N_MELS, None)  # expected (n_mels, T). None = any T

TARGET_LABELS = set(globals().get("TARGET_LABELS", {
    'Yellow->red card','Ball out of play','Foul','Indirect free-kick','Penalty','Shots off target',
    'Offside','Yellow card','Red card','Goal','Direct free-kick','Substitution','Clearance',
    'Kick-off','Corner','Throw-in','Shots on target'
}))

SEASON_RE = re.compile(r"^\d{4}-\d{4}$")

def _parse_time_mmss(s: str):
    parts = s.split(":")
    try:
        if len(parts)==3: hh,mm,ss = parts; return int(hh)*3600 + int(mm)*60 + float(ss)
        if len(parts)==2: mm,ss = parts; return int(mm)*60 + float(ss)
    except Exception: return None
    return None

def parse_comp_season_from_game_dir(game_dir: Path, root: Path):
    parts = game_dir.relative_to(root).parts
    comp = None; season = None
    for i, p in enumerate(parts):
        if p == TARGET_COMP:
            comp = p
            if i+1 < len(parts) and SEASON_RE.match(parts[i+1]): season = parts[i+1]
            break
    if comp is None and len(parts) >= 1: comp = parts[0]
    if season is None:
        for p in parts:
            if SEASON_RE.match(p): season = p; break
    return comp, season

def find_label_game_dirs(root: Path):
    return [p.parent for p in root.rglob("Labels*.json")]

def load_labels_with_half(labels_path: Path):
    with open(labels_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    ann = data.get("annotations") or data.get("events") or data.get("labels") or []
    out=[]
    for ev in ann:
        lab  = ev.get("label") or ev.get("event") or ev.get("type")
        if not lab: continue
        half = ev.get("half")
        if half not in (1,2):
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                try: half = int(gt.split(" - ")[0].strip())
                except: half = 1
        pos = ev.get("position")
        t = float(pos) if isinstance(pos,(int,float)) else (_parse_time_mmss(pos) if isinstance(pos,str) else None)
        if t is None:
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                t = _parse_time_mmss(gt.split(" - ")[1].strip())
        if t is None: continue
        out.append({"time_s": float(t), "label": lab, "half": int(half)})
    return out

def expected_half_wav_path(comp: str, season: str, game: str, half: int, sr: int = TARGET_SR) -> Path:
    return AUDIO_DIR / f"{comp}__{season}__{game}__H{half}_{sr}Hz.wav"

def slugify(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","-", s.strip().lower()).strip("-")

# ---- Atomic save helpers ----
def atomic_save_npy(path: Path, arr: np.ndarray):
    path.parent.mkdir(parents=True, exist_ok=True)
    with tempfile.NamedTemporaryFile(delete=False, dir=str(path.parent), suffix=".tmp") as tf:
        tmp = Path(tf.name)
    try:
        np.save(tmp, arr, allow_pickle=False)
        os.replace(tmp, path)  # atomic on Windows/Unix
    finally:
        if tmp.exists():
            try: tmp.unlink()
            except: pass

def atomic_save_wav(path: Path, tensor: torch.Tensor, sr: int):
    path.parent.mkdir(parents=True, exist_ok=True)
    with tempfile.NamedTemporaryFile(delete=False, dir=str(path.parent), suffix=".tmp.wav") as tf:
        tmp = Path(tf.name)
    try:
        torchaudio.save(str(tmp), tensor, sr)
        os.replace(tmp, path)
    finally:
        if tmp.exists():
            try: tmp.unlink()
            except: pass

def is_valid_npy(path: Path, expected_first_dim: int | None = None) -> bool:
    try:
        if not path.exists() or path.stat().st_size == 0:
            return False
        arr = np.load(path, mmap_mode="r")
        if arr.ndim != 2:
            return False
        if expected_first_dim is not None and arr.shape[0] != expected_first_dim:
            return False
        return True
    except Exception:
        return False

# ---------- 1) Collect EPL events with existing half WAV ----------
label_games = [gd for gd in find_label_game_dirs(SN_ROOT) if parse_comp_season_from_game_dir(gd, SN_ROOT)[0] == TARGET_COMP]
rows=[]
for gd in tqdm(label_games, desc="Index labels"):
    comp, season = parse_comp_season_from_game_dir(gd, SN_ROOT)
    game = gd.name
    labels_json = (gd / "Labels-v2.json") if (gd / "Labels-v2.json").exists() else (gd / "Labels.json")
    evs = load_labels_with_half(labels_json)
    for e in evs:
        if e["label"] not in TARGET_LABELS: continue
        wav_p = expected_half_wav_path(comp, season, game, e["half"], TARGET_SR)
        if wav_p.exists():
            rows.append({
                "competition": comp, "season": season, "game": game, "game_dir": str(gd),
                "event_time_s": e["time_s"], "event_label": e["label"], "half": e["half"],
                "audio_wav": str(wav_p)
            })
df_events = pd.DataFrame(rows).drop_duplicates(subset=["game_dir","event_time_s","event_label","half"]).reset_index(drop=True)
print(f"[Events] Kept (labels + half WAV exists): {len(df_events)}")
if df_events.empty:
    raise SystemExit("No events with matching half WAVs found. Put some WAVs in full_match_audio_wav/.")

# ---------- 2) Mel frontend ----------
mel_tf = torchaudio.transforms.MelSpectrogram(
    sample_rate=TARGET_SR, n_fft=N_FFT, hop_length=HOP, win_length=N_FFT,
    n_mels=N_MELS, f_min=0.0, f_max=TARGET_SR/2.0, center=True, pad_mode="reflect", power=2.0
)
amp2db = torchaudio.transforms.AmplitudeToDB(stype="power")

def load_half_wav_cached(wav_path: str, cache: dict) -> torch.Tensor:
    if wav_path in cache: return cache[wav_path]
    wav, sr = torchaudio.load(wav_path)
    if wav.shape[0] > 1: wav = wav.mean(dim=0, keepdim=True)
    if sr != TARGET_SR:  wav = torchaudio.functional.resample(wav, sr, TARGET_SR)
    cache[wav_path] = wav.squeeze(0).contiguous()
    return cache[wav_path]

def cut_centered(signal_1d: torch.Tensor, sr: int, center_s: float, win_s: float) -> torch.Tensor:
    total = signal_1d.numel()
    n_win = int(round(win_s * sr))
    half = n_win // 2
    center = int(round(center_s * sr))
    start = center - half
    end   = start + n_win
    pad_left = max(0, -start)
    pad_right = max(0, end - total)
    if pad_left or pad_right:
        sig = torch.nn.functional.pad(signal_1d, (pad_left, pad_right))
        start = start + pad_left
        end   = start + n_win
        clip = sig[start:end]
    else:
        start = max(0, start); end = min(total, end)
        clip = signal_1d[start:end]
    if clip.numel() != n_win:
        clip = torch.nn.functional.pad(clip, (0, n_win - clip.numel()))
    return clip

# Ensure folders exist
for w in WINDOWS:
    (CLIPS_DIR / f"win{w}s").mkdir(parents=True, exist_ok=True)
    (FEAT_DIR  / f"win{w}s").mkdir(parents=True, exist_ok=True)

# ---------- 3) Generate clips + mels + CSV (skip existing + repair corrupt) ----------
rows_out = []
wav_cache = {}
labels_sorted = sorted(df_events["event_label"].unique())
label_to_id = {lab:i for i,lab in enumerate(labels_sorted)}

pbar = tqdm(df_events.itertuples(index=False), total=len(df_events), desc="Make/collect clips + mels")
for r in pbar:
    comp, season, game = r.competition, r.season, r.game
    half, t_s = int(r.half), float(r.event_time_s)
    label     = r.event_label
    label_id  = label_to_id[label]
    half_wav  = r.audio_wav

    # Decide which windows actually need recompute (missing or corrupt or overwrite)
    plan_windows = []
    for win_s in WINDOWS:
        stem = f"{comp}__{season}__{game}__H{half}__{slugify(label)}__t{int(round(t_s*1000))}ms__w{win_s}s"
        wav_out  = CLIPS_DIR / f"win{win_s}s" / f"{stem}.wav"
        mel_out  = FEAT_DIR  / f"win{win_s}s" / f"{stem}.npy"

        mel_exists = mel_out.exists()
        mel_valid  = is_valid_npy(mel_out, expected_first_dim=VALIDATE_NPY_SHAPE[0]) if mel_exists else False
        need_mel   = OVERWRITE_MELS or (not mel_exists) or (REPAIR_CORRUPT and not mel_valid)

        if need_mel:
            plan_windows.append((win_s, stem, wav_out, mel_out))

    # If nothing to do, just consume existing valid mels
    if not plan_windows:
        for win_s in WINDOWS:
            stem = f"{comp}__{season}__{game}__H{half}__{slugify(label)}__t{int(round(t_s*1000))}ms__w{win_s}s"
            wav_out  = CLIPS_DIR / f"win{win_s}s" / f"{stem}.wav"
            mel_out  = FEAT_DIR  / f"win{win_s}s" / f"{stem}.npy"
            if is_valid_npy(mel_out, expected_first_dim=N_MELS):
                mel = np.load(mel_out, mmap_mode="r")
                rows_out.append({
                    "competition": comp, "season": season, "game": game, "half": half,
                    "t_center_s": t_s, "win_s": win_s, "event_label": label, "label_id": label_id,
                    "audio_wav": str(wav_out), "mel_npy": str(mel_out),
                    "frames": int(mel.shape[1]), "n_mels": int(mel.shape[0]),
                })
        continue

    # Work needed → load half once
    full = load_half_wav_cached(half_wav, wav_cache)

    # Compute/rewrite for the windows that need it
    for (win_s, stem, wav_out, mel_out) in plan_windows:
        clip = cut_centered(full, TARGET_SR, t_s, win_s)

        if OVERWRITE_WAVS or (not wav_out.exists()):
            atomic_save_wav(wav_out, clip.unsqueeze(0), TARGET_SR)

        with torch.no_grad():
            mel = mel_tf(clip.unsqueeze(0))
            mel_db = amp2db(mel).squeeze(0).cpu().numpy()
            mu, sd = mel_db.mean(), mel_db.std() + 1e-8
            mel_db = (mel_db - mu) / sd

        atomic_save_npy(mel_out, mel_db)

    # Append rows for all windows (now guaranteed valid)
    for win_s in WINDOWS:
        stem = f"{comp}__{season}__{game}__H{half}__{slugify(label)}__t{int(round(t_s*1000))}ms__w{win_s}s"
        wav_out  = CLIPS_DIR / f"win{win_s}s" / f"{stem}.wav"
        mel_out  = FEAT_DIR  / f"win{win_s}s" / f"{stem}.npy"
        if is_valid_npy(mel_out, expected_first_dim=N_MELS):
            mel = np.load(mel_out, mmap_mode="r")
            rows_out.append({
                "competition": comp, "season": season, "game": game, "half": half,
                "t_center_s": t_s, "win_s": win_s, "event_label": label, "label_id": label_id,
                "audio_wav": str(wav_out), "mel_npy": str(mel_out),
                "frames": int(mel.shape[1]), "n_mels": int(mel.shape[0]),
            })

# ---------- 4) Merge/build index ----------
new_index = pd.DataFrame(rows_out).sort_values(
    ["season","game","half","t_center_s","win_s"]
).reset_index(drop=True)

if INDEX_CSV.exists():
    old_index = pd.read_csv(INDEX_CSV)
    combo = pd.concat([old_index, new_index], ignore_index=True)
    combo.drop_duplicates(subset=["mel_npy"], inplace=True)
    combo.sort_values(["season","game","half","t_center_s","win_s"], inplace=True)
    combo.reset_index(drop=True, inplace=True)
    combo.to_csv(INDEX_CSV, index=False)
    print(f"[Done] Updated index → {INDEX_CSV} (old {len(old_index)} + new {len(new_index)} → unique {len(combo)})")
    display(combo.head(5))
else:
    new_index.to_csv(INDEX_CSV, index=False)
    print(f"[Done] Wrote {len(new_index)} rows → {INDEX_CSV}")
    display(new_index.head(5))

Index labels: 100%|██████████| 190/190 [00:01<00:00, 107.97it/s]


[Events] Kept (labels + half WAV exists): 3969


Make/collect clips + mels:   2%|▏         | 94/3969 [00:00<00:19, 198.62it/s]C:\Users\CYBORG 15\AppData\Roaming\Python\Python312\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
Make/collect clips + mels: 100%|██████████| 3969/3969 [00:30<00:00, 131.00it/s]


[Done] Updated index → sn_audio_work\dataset_index.csv (old 7938 + new 7937 → unique 7938)


,competition,season,game,half,t_center_s,win_s,event_label,label_id,audio_wav,mel_npy,frames,n_mels
0,england_epl,2014-2015,2015-02-21 - 18-00 Chelsea 1 - 1 Burnley,1,0.0,15,Kick-off,7,sn_audio_work\clips\win15s\england_epl__2014-2...,sn_audio_work\mel64\win15s\england_epl__2014-2...,751,64
1,england_epl,2014-2015,2015-02-21 - 18-00 Chelsea 1 - 1 Burnley,1,0.0,30,Kick-off,7,sn_audio_work\clips\win30s\england_epl__2014-2...,sn_audio_work\mel64\win30s\england_epl__2014-2...,1501,64
2,england_epl,2014-2015,2015-02-21 - 18-00 Chelsea 1 - 1 Burnley,1,133.0,15,Ball out of play,0,sn_audio_work\clips\win15s\england_epl__2014-2...,sn_audio_work\mel64\win15s\england_epl__2014-2...,751,64
3,england_epl,2014-2015,2015-02-21 - 18-00 Chelsea 1 - 1 Burnley,1,133.0,30,Ball out of play,0,sn_audio_work\clips\win30s\england_epl__2014-2...,sn_audio_work\mel64\win30s\england_epl__2014-2...,1501,64
4,england_epl,2014-2015,2015-02-21 - 18-00 Chelsea 1 - 1 Burnley,1,149.0,15,Throw-in,14,sn_audio_work\clips\win15s\england_epl__2014-2...,sn_audio_work\mel64\win15s\england_epl__2014-2...,751,64


**PART 2: Baseline Model**

In [1]:
# === RCNN baseline: setup ===
import os, math, json, time, random, re, gc
from pathlib import Path
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.metrics import classification_report, f1_score, accuracy_score

# Paths (re-use your earlier ones)
WORK_DIR  = Path("./sn_audio_work")
INDEX_CSV = WORK_DIR / "dataset_index.csv"  # created by your mel feature cell

# Repro & device
SEED = 1337
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Training config
WIN_S            = 30       # choose 15 or 30 to match what you saved
BATCH_SIZE       = 16
NUM_EPOCHS       = 20
LR               = 2e-4
WEIGHT_DECAY     = 1e-4
LABEL_SMOOTHING  = 0.05
USE_MIXED_PREC   = True
NUM_WORKERS      = 2

# SpecAugment-lite
AUG_FREQ_MASKS   = 1   # 0 to disable
AUG_FREQ_WIDTH   = 8
AUG_TIME_MASKS   = 1
AUG_TIME_WIDTH   = 24  # frames


C:\Users\CYBORG 15\AppData\Roaming\Python\Python312\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\utils\__init__.py:16: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.3)
  from scipy.sparse import issparse


Device: cuda


In [2]:
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("build cuda:", torch.version.cuda)
print("device count:", torch.cuda.device_count())
if torch.cuda.is_available(): print("gpu0:", torch.cuda.get_device_name(0))

torch: 2.6.0+cu124
cuda available: True
build cuda: 12.4
device count: 1
gpu0: NVIDIA GeForce RTX 4050 Laptop GPU


In [3]:
# === Load dataset index and split ===
df = pd.read_csv(INDEX_CSV)
df = df[df["win_s"] == WIN_S].copy()

# Only valid rows with existing feature files
df = df[df["mel_npy"].apply(lambda p: Path(p).exists())].reset_index(drop=True)

# Classes
labels_sorted = sorted(df["event_label"].unique())
label_to_id   = {lab:i for i, lab in enumerate(labels_sorted)}
id_to_label   = {i:lab for lab,i in label_to_id.items()}
num_classes   = len(labels_sorted)
print("Classes:", num_classes, labels_sorted[:10], "...")

# Encode labels
df["y"] = df["event_label"].map(label_to_id)

# Split by game (season+game combo) to reduce leakage
df["game_key"] = df["season"].astype(str) + "/" + df["game"].astype(str)
games = df["game_key"].unique().tolist()
random.shuffle(games)

n_train = int(0.8 * len(games))
train_games = set(games[:n_train])
val_games   = set(games[n_train:])

train_df = df[df["game_key"].isin(train_games)].reset_index(drop=True)
val_df   = df[df["game_key"].isin(val_games)].reset_index(drop=True)

print(f"Train clips: {len(train_df)}  |  Val clips: {len(val_df)}  | Games train/val: {len(train_games)}/{len(val_games)}")


Classes: 16 ['Ball out of play', 'Clearance', 'Corner', 'Direct free-kick', 'Foul', 'Goal', 'Indirect free-kick', 'Kick-off', 'Offside', 'Penalty'] ...
Train clips: 3172  |  Val clips: 797  | Games train/val: 31/8


In [4]:
# === Dataset & collate ===
class MelClipDataset(Dataset):
    """
    Expects mel_npy shaped [n_mels, T], z-scored per clip (as created earlier).
    Applies light SpecAugment on-the-fly (optional).
    """
    def __init__(self, frame, train=True, aug_freq_masks=0, aug_freq_w=8, aug_time_masks=0, aug_time_w=24):
        self.frame = frame
        self.train = train
        self.aug_freq_masks = aug_freq_masks if train else 0
        self.aug_freq_w     = aug_freq_w
        self.aug_time_masks = aug_time_masks if train else 0
        self.aug_time_w     = aug_time_w

    def __len__(self):
        return len(self.frame)

    def _spec_augment(self, m):
        # m: [n_mels, T]
        n_mels, T = m.shape
        out = m.copy()

        # Frequency masks
        for _ in range(self.aug_freq_masks):
            w = random.randint(1, min(self.aug_freq_w, n_mels // 2))
            f0 = random.randint(0, max(0, n_mels - w))
            out[f0:f0+w, :] = 0.0

        # Time masks
        for _ in range(self.aug_time_masks):
            w = random.randint(1, min(self.aug_time_w, T // 2))
            t0 = random.randint(0, max(0, T - w))
            out[:, t0:t0+w] = 0.0

        return out

    def __getitem__(self, idx):
        r = self.frame.iloc[idx]
        mel = np.load(r["mel_npy"])  # [n_mels, T] float32-ish
        if self.train:
            mel = self._spec_augment(mel)
        # to tensor [C=1, n_mels, T]
        x = torch.from_numpy(mel).float().unsqueeze(0)
        y = int(r["y"])
        meta = {
            "competition": r["competition"],
            "season": r["season"],
            "game": r["game"],
            "half": int(r["half"]),
            "t_center_s": float(r["t_center_s"]),
            "label": r["event_label"],
        }
        return x, y, meta

def pad_time_collate(batch):
    """
    Pad variable T to max_T in the batch on the right; returns:
    x: [B, 1, n_mels, T_max], y: [B], lengths: [B]
    """
    xs, ys, metas = zip(*batch)
    n_mels = xs[0].shape[1]
    lengths = torch.tensor([t.shape[-1] for t in xs], dtype=torch.long)
    T_max = int(lengths.max())

    x_padded = torch.zeros(len(xs), 1, n_mels, T_max, dtype=torch.float32)
    for i, t in enumerate(xs):
        T = t.shape[-1]
        x_padded[i, :, :, :T] = t

    y = torch.tensor(ys, dtype=torch.long)
    return x_padded, y, lengths, metas

train_ds = MelClipDataset(train_df, train=True,
                          aug_freq_masks=AUG_FREQ_MASKS, aug_freq_w=AUG_FREQ_WIDTH,
                          aug_time_masks=AUG_TIME_MASKS, aug_time_w=AUG_TIME_WIDTH)
val_ds   = MelClipDataset(val_df,   train=False)

# Weighted sampler for class imbalance
class_counts = train_df["y"].value_counts().reindex(range(num_classes), fill_value=0).values
class_weights = 1.0 / (class_counts + 1e-6)
sample_weights = train_df["y"].map(lambda yi: class_weights[yi]).values
sampler = WeightedRandomSampler(weights=torch.from_numpy(sample_weights).float(),
                                num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler,
                          num_workers=NUM_WORKERS, pin_memory=True, collate_fn=pad_time_collate)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True, collate_fn=pad_time_collate)

print("Loaders ready.")


Loaders ready.


In [5]:
# === RCNN model ===
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, pool=(2,2), p_drop=0.1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
        self.pool = nn.MaxPool2d(pool)
        self.drop = nn.Dropout(p_drop)
    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        x = self.drop(x)
        return x

class RCNN_Audio(nn.Module):
    """
    Input: [B, 1, n_mels, T]
    Conv downsamples in freq/time; then BiGRU over time; then pool and classify.
    """
    def __init__(self, n_mels, n_classes, rnn_hidden=128, rnn_layers=1, use_attention=False):
        super().__init__()
        self.use_attention = use_attention

        # 2 lightweight conv blocks (keep it small for your GPU)
        self.feat = nn.Sequential(
            ConvBlock(1, 32, pool=(2,2), p_drop=0.1),   # -> [B,32,n_mels/2,T/2]
            ConvBlock(32, 64, pool=(2,2), p_drop=0.1),  # -> [B,64,n_mels/4,T/4]
        )

        # Project freq dimension to a feature dim for RNN
        self.proj = nn.Conv2d(64, 128, kernel_size=(n_mels//4, 1))  # collapse freq
        # Now shape: [B,128,1,T’] -> squeeze freq -> [B,128,T’]
        
        self.rnn = nn.GRU(input_size=128, hidden_size=rnn_hidden, num_layers=rnn_layers,
                          batch_first=True, dropout=0.0, bidirectional=True)

        if use_attention:
            self.attn = nn.Linear(2*rnn_hidden, 1)

        self.head = nn.Sequential(
            nn.LayerNorm(2*rnn_hidden),
            nn.Linear(2*rnn_hidden, n_classes)
        )

    def forward(self, x, lengths=None):
        # x: [B, 1, n_mels, T]
        B = x.size(0)
        x = self.feat(x)                           # [B,64,n_mels/4,T’]
        x = self.proj(x)                           # [B,128,1,T’]
        x = x.squeeze(2).transpose(1,2)            # [B,T’,128]

        # pack if lengths given (convert time to downsampled lengths)
        if lengths is not None:
            # two MaxPool(2,2) along time -> /4 in time
            ds_lengths = (lengths.float() / 4.0).floor().clamp(min=1).long()
            x_packed = nn.utils.rnn.pack_padded_sequence(x, ds_lengths.cpu(), batch_first=True, enforce_sorted=False)
            x_rnn, _ = self.rnn(x_packed)
            x, _ = nn.utils.rnn.pad_packed_sequence(x_rnn, batch_first=True)
        else:
            x, _ = self.rnn(x)

        if self.use_attention:
            # simple attention over time
            attn_w = self.attn(x).squeeze(-1)                 # [B,T’]
            attn_w = torch.softmax(attn_w, dim=1)
            out = torch.sum(x * attn_w.unsqueeze(-1), dim=1)  # [B, 2H]
        else:
            # mean pool over time
            out = x.mean(dim=1)                                # [B, 2H]

        logits = self.head(out)
        return logits

# Instantiate
n_mels = int(train_df.iloc[0]["n_mels"])
model = RCNN_Audio(n_mels=n_mels, n_classes=num_classes, rnn_hidden=128, rnn_layers=1, use_attention=False).to(device)
sum(p.numel() for p in model.parameters())/1e6


0.399344

In [ ]:
# === Train / Validate ===
class SmoothCELoss(nn.Module):
    def __init__(self, label_smoothing=0.0, class_weights=None):
        super().__init__()
        self.ls = float(label_smoothing)
        self.register_buffer("w", None if class_weights is None else torch.tensor(class_weights, dtype=torch.float32))
    def forward(self, logits, target):
        if self.ls > 0:
            n = logits.size(-1)
            log_probs = F.log_softmax(logits, dim=-1)
            with torch.no_grad():
                true_dist = torch.zeros_like(log_probs)
                true_dist.fill_(self.ls / (n - 1))
                true_dist.scatter_(1, target.unsqueeze(1), 1.0 - self.ls)
            if self.w is not None:
                # apply class weights as per-target scaling
                weight = self.w[target]
                loss = -(true_dist * log_probs).sum(dim=1) * weight
                return loss.mean()
            else:
                return -(true_dist * log_probs).sum(dim=1).mean()
        else:
            return F.cross_entropy(logits, target, weight=self.w)

# class weights from train set
train_counts = train_df["y"].value_counts().reindex(range(num_classes), fill_value=0).values.astype(np.float32)
class_weights = (train_counts.sum() / (train_counts + 1e-6))
class_weights = class_weights / class_weights.mean()

criterion = SmoothCELoss(label_smoothing=LABEL_SMOOTHING, class_weights=class_weights).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

scaler = torch.cuda.amp.GradScaler(enabled=(USE_MIXED_PREC and device.type=="cuda"))

def run_epoch(loader, train=True):
    model.train(train)
    total, correct = 0, 0
    y_true, y_pred = [], []
    running_loss = 0.0

    for x, y, lengths, metas in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        with torch.set_grad_enabled(train):
            with torch.cuda.amp.autocast(enabled=(USE_MIXED_PREC and device.type=="cuda")):
                logits = model(x, lengths=lengths)
                loss = criterion(logits, y)

            if train:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)

        running_loss += loss.item() * x.size(0)
        probs = torch.softmax(logits.detach(), dim=-1)
        pred = probs.argmax(dim=-1)

        total += x.size(0)
        correct += (pred == y).sum().item()
        y_true.extend(y.cpu().tolist())
        y_pred .extend(pred.cpu().tolist())

    epoch_loss = running_loss / max(1, total)
    acc = correct / max(1, total)
    f1m = f1_score(y_true, y_pred, average="macro", zero_division=0)
    return epoch_loss, acc, f1m, (y_true, y_pred)

best_f1 = 0.0
best_path = WORK_DIR / f"rcnn_baseline_win{WIN_S}s.pt"

for epoch in range(1, NUM_EPOCHS+1):
    tr_loss, tr_acc, tr_f1, _ = run_epoch(train_loader, train=True)
    va_loss, va_acc, va_f1, (vy, vp) = run_epoch(val_loader,   train=False)
    scheduler.step()

    if va_f1 > best_f1:
        best_f1 = va_f1
        torch.save({"model": model.state_dict(),
                    "labels_sorted": labels_sorted,
                    "cfg": {"win_s": WIN_S, "n_mels": n_mels}}, best_path)

    print(f"[{epoch:02d}/{NUM_EPOCHS}] "
          f"train: loss={tr_loss:.4f} acc={tr_acc:.3f} f1={tr_f1:.3f} | "
          f"val: loss={va_loss:.4f} acc={va_acc:.3f} f1={va_f1:.3f} | best_f1={best_f1:.3f}")

print("Saved best →", best_path)
print(classification_report(vy, vp, target_names=[id_to_label[i] for i in range(num_classes)], digits=3, zero_division=0))


C:\Users\CYBORG 15\AppData\Local\Temp\ipykernel_33336\750936264.py:34: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(USE_MIXED_PREC and device.type=="cuda"))


In [1]:
# === Inference helpers ===
@torch.no_grad()
def predict_mel_npy(model, npy_path):
    mel = np.load(npy_path)                      # [n_mels, T]
    x = torch.from_numpy(mel).float().unsqueeze(0).unsqueeze(0).to(device)  # [1,1,n_mels,T]
    logits = model(x, lengths=torch.tensor([mel.shape[1]])).softmax(dim=-1).squeeze(0).cpu().numpy()
    pred_id = int(logits.argmax())
    return pred_id, logits

def load_best_model(path):
    ckpt = torch.load(path, map_location=device)
    m = RCNN_Audio(n_mels=ckpt["cfg"]["n_mels"], n_classes=len(ckpt["labels_sorted"]),
                   rnn_hidden=128, rnn_layers=1, use_attention=False).to(device)
    m.load_state_dict(ckpt["model"]); m.eval()
    return m, ckpt["labels_sorted"]

# Example:
# best_model, class_names = load_best_model(best_path)
# test_row = val_df.sample(1, random_state=SEED).iloc[0]
# pid, probs = predict_mel_npy(best_model, test_row["mel_npy"])
# print("GT:", test_row["event_label"], "Pred:", class_names[pid])


NameError: name 'torch' is not defined